In [1]:
import pandas as pd

In [ ]:
#examine current cross_walk file
#calm = pd.read_csv("data/geo_cross_walk.csv")
#calm.head(2)

In [13]:
#read semcog block GIS file
sem = pd.read_csv("data/semcog_block2010.txt") 
sembg = sem.loc[sem.SUBTYPECD == 2][['GEOID10']]
print(sembg.dtypes)
sembg['GEOID10'].head(2)

GEOID10    int64
dtype: object


2086    261635647002
2087    261635032001
Name: GEOID10, dtype: int64

In [14]:
sembg['BLKGRPID'] = sembg['GEOID10'].apply(int) #numeric full BG id
sembg['TRACTID'] = sembg['GEOID10'].apply(str).str[:11].apply(int) #numeric full tract id
sembg['TRACT'] = sembg['GEOID10'].apply(str).str[5:11].apply(int)
sembg['BLKGRP'] = sembg['GEOID10'].apply(str).str[11].apply(int)


In [15]:
# read 2010 tract to puma file from Census
tract_puma = pd.read_csv("data/2010_Census_Tract_to_2010_PUMA.txt")
tract_puma = tract_puma.loc[tract_puma.STATEFP == 26]
tract_puma['TRACTID'] = tract_puma['STATEFP'] * 1000000000 + tract_puma['COUNTYFP'] * 1000000 + tract_puma['TRACTCE']
tract_puma.head(2)

,STATEFP,COUNTYFP,TRACTCE,PUMA5CE,TRACTID
32693,26,1,100,300,26001000100
32694,26,1,970100,300,26001970100


In [16]:
tract_puma.loc[tract_puma.duplicated(subset=['TRACTID'])] #verify if tract id are unique
#semblk.loc[semblk.duplicated(subset=['TRACTID'])]

,STATEFP,COUNTYFP,TRACTCE,PUMA5CE,TRACTID


In [17]:
# merge block group file with puma file
semblkgrp = pd.merge(sembg, tract_puma, left_on = 'TRACTID', right_on = 'TRACTID', how = 'left')


In [18]:
semblkgrp.dtypes

GEOID10     int64
BLKGRPID    int64
TRACTID     int64
TRACT       int64
BLKGRP      int64
STATEFP     int64
COUNTYFP    int64
TRACTCE     int64
PUMA5CE     int64
dtype: object

In [20]:
semblkgrp.head()

,GEOID10,BLKGRPID,TRACTID,TRACT,BLKGRP,STATEFP,COUNTYFP,TRACTCE,PUMA,REGION
0,261635647002,261635647002,26163564700,564700,2,26,163,564700,3201,4
1,261635032001,261635032001,26163503200,503200,1,26,163,503200,3210,4
2,261635020002,261635020002,26163502000,502000,2,26,163,502000,3211,4
3,261635807001,261635807001,26163580700,580700,1,26,163,580700,3207,4
4,261635342003,261635342003,26163534200,534200,3,26,163,534200,3208,4


In [19]:
semblkgrp['REGION'] = 4
semblkgrp.rename(columns = {'PUMA5CE': 'PUMA'}, inplace = True)

In [23]:
#semblkgrp.to_csv("data/sem_geo_cross_walk.csv")

In [24]:
oakblkgrp = semblkgrp.loc[semblkgrp.COUNTYFP == 125 ]
oakblkgrp.to_csv("data/oak_geo_cross_walk.csv")